In [2]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue
import tqdm
# from src import data 
# from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import cufflinks as cf
# cf.go_offline()

sys.path.append(os.path.abspath('/home/hengtian/lucy/PortfolioOptimizer/'))
from PortfolioOptimizerMaster.src import tools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [3]:
##每日换手率
tov = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/turnover_rate.csv", index_col = 0)
tov = tov['2016':'2021-07-05']

In [3]:
#变化率
tov_pct = pd.read_csv("/home/hengtian/kate/turnover/data/pct_tov.csv", index_col = 0)

In [4]:
#稳定度
tov_str = pd.read_csv("/home/hengtian/kate/turnover/data/str_tov.csv", index_col = 0)

In [5]:
#传统换手率20日
tov_trad = pd.read_csv("/home/hengtian/kate/turnover/data/trad_tov.csv", index_col = 0)

In [6]:
#传统波动率20日
vol = pd.read_csv("/home/hengtian/kate/turnover/data/vol_20.csv", index_col = 0)

In [42]:
bd = tools.BasicData(bd_dir="/home/hengtian/lucy/factor_service/basic_data/", reload=False)

[finish] loading factor_data from the factor_data_cache.pkl
[finish] loading comp_data from the comp_data_cache.pkl
[finish] loading basic data 


In [62]:
comp_dict = {
            'tov20': bd.factor_data['Tov_20D_Avg'],
             'size': bd.factor_data['NegMktCap'], 
             'atr20': bd.factor_data['TR_20D_Avg'],
             'mom20': bd.factor_data['Mom_20D'],
            }

In [48]:
trad_tov_zscore = tov_trad.apply(extreme_median, axis=1).apply(zscore, axis=1) # 标准化
trad_tov_zscore.index = pd.to_datetime(trad_tov_zscore.index).date

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [45]:
str_zscore = tov_str.apply(extreme_median, axis=1).apply(zscore, axis=1) # 标准化
str_zscore.index = pd.to_datetime(str_zscore.index).date

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """
/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [46]:
pct_zscore = tov_pct.apply(extreme_median, axis=1).apply(zscore, axis=1) # 标准化
pct_zscore.index = pd.to_datetime(pct_zscore.index).date

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [47]:
vol_zscore=vol.apply(extreme_median, axis=1).apply(zscore, axis=1) # 标准化
vol_zscore.index = pd.to_datetime(vol_zscore.index).date

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """


In [7]:
def vol_20(stock,q):
    """
    传统20日波动率因子
    
    """
    i = 20
    p=inday[stock]
    stat = np.full(len(p),np.nan)
    while i < len(p):
        sub = p.iloc[i-20:i]
        if len(sub.dropna()) != 20:
            i+=1
        else:
            stat[i]=sub.std()
            i+=1
#     print(stat)
    return q.put({stock:stat})

In [66]:
def trad_tov(stock,q):
    """
    传统换手率20日
    """
    i = 20
    p = tov[stock]
    stat = np.full(len(tov),np.nan)
    while i < len(tov):
        sub = p.iloc[i-20:i]
        if len(sub.dropna()) !=20:
            i+=1
        else:
            stat[i] = sub.mean()
            i+=1
    return q.put({stock:stat})
#     return stat

In [26]:
def pct_tov(stock,q):
    """
    换手率变化率20日
    
    """
    i =60
#     j =40
    p=tov[stock]
    stat = np.full(len(p),np.nan)
    while i<len(p):
        if len(p.iloc[i-20:i].dropna()) !=20:
            i+=1
        else:
            base = p.iloc[i-60:i-20].mean(skipna =True)
#             print(p.iloc[i-60:i-20])
            pct =p.iloc[i-20:i]/base -1
#             print(p.iloc[i-20:i])
            stat[i] =pct.mean()
#             j+=1
            i+=1
    return q.put({stock:stat})
#     return stat

In [36]:
def str_pct(stock,q):
    """
    换手变化率的稳定度20日
    
    """
    i =60
    p=tov_pct[stock]
    stat = np.full(len(p),np.nan)
    while i<len(p):
        if len(p.iloc[i-20:i].dropna()) !=20:
            i+=1
        else:
            stat[i] =p.iloc[i-20:i].std()
            i+=1
    return q.put({stock:stat})
    

In [13]:
def str_tov(stock,q):
    """
    量稳换手率
    
    """
    i = 20
    p = tov[stock]
    sta = np.full(len(p),np.nan)
    while i< len(p):
        if len(p.iloc[i-20:i].dropna())!=20:
            i+=1
        else:
            sta[i] =p.iloc[i-20:i].std()
            i+=1
    return q.put({stock:sta})

In [30]:
def std_mean(stock,q):
    """
    稳定度/传统
    """
    
    i = 20
    p = tov[stock]
    sta = np.full(len(p),np.nan)
    while i <len(p):
        if len(p.iloc[i-20:i].dropna())!=20:
            i+=1
        else:
            mean =p.iloc[i-20:i].mean()
            std = p.iloc[i-20:i].std()
            sta[i] =std/mean
#             print(mean)
#             print(std)
#             print(mean/std)
            i+=1
#     return sta        
    return q.put({stock:sta})

In [8]:
def del_vol(stock,q):
    """
    STR 剔除波动率20日
    """
    s_str = tov_str[stock]
    s_str = (s_str - np.nanmean(s_str)) / np.nanstd(s_str)
    s_vol = vol[stock]
    s_vol = (s_vol-np.nanmean(s_vol))/np.nanstd(s_vol)
#     print(s_str.shape)
    idx_a =s_str[s_str.isnull()].index
    idx_b = s_vol[s_vol.isnull()].index
    total = list(idx_a.union(idx_b))
    if len(total) == 1339:
        resid = np.full(len(total),np.nan)
#     print(total)
    else:
        s_str[total] = np.nan
        s_vol[total]=np.nan
    #     print(s_vol.shape)
        rg = LinearRegression().fit(np.array(s_vol.dropna()).reshape(-1,1),s_str.dropna())
        resid =s_str.values-(s_vol*rg.coef_+rg.intercept_)
        resid = resid.values
        print(resid)
#     return resid
    return q.put({stock:resid})

In [71]:
comp_dict ={
            'mom20': bd.factor_data['Mom_20D']
}

In [72]:
d = calc_residual(trad_tov_zscore,comp_dict)
d.to_csv("/home/hengtian/kate/turnover/data/trad_del_mom.csv")

20it [00:00, 193.35it/s]

time period: 2016-01-04 ~ 2021-07-05
missing all data: 2016-01-04
missing all data: 2016-01-05
missing all data: 2016-01-06
missing all data: 2016-01-07
missing all data: 2016-01-08
missing all data: 2016-01-11
missing all data: 2016-01-12
missing all data: 2016-01-13
missing all data: 2016-01-14
missing all data: 2016-01-15
missing all data: 2016-01-18
missing all data: 2016-01-19
missing all data: 2016-01-20
missing all data: 2016-01-21
missing all data: 2016-01-22
missing all data: 2016-01-25
missing all data: 2016-01-26
missing all data: 2016-01-27
missing all data: 2016-01-28
missing all data: 2016-01-29


1339it [00:06, 198.11it/s]


In [52]:
d.to_csv("trad_del_atr20.csv")

In [25]:
def calc_residual(df, comp_dict):
    """
    计算截面 residual
    y = df
    x = df_l
    """
    res_residual = {}
    model = LinearRegression()
    start_date = df.index.min()
    end_date = df.index.max()
    for key, df_tmp in comp_dict.items():
        start_date = max(start_date, df_tmp.index.min())
        end_date = min(end_date, df_tmp.index.max())
    
    df = df.loc[start_date:end_date,:]
    print("time period: {} ~ {}".format(start_date, end_date))

    for date, row in tqdm.tqdm(df.iterrows()):
        tmp_l = []
        tmp_l.append(df.loc[date])
        for key, df_tmp in comp_dict.items():
            tmp_l.append(df_tmp.loc[date])
        concat = pd.concat(tmp_l, axis=1).dropna(how="any", axis=0)
        if len(concat) == 0:
            print("missing all data: {}".format(date))
            continue
        y = concat.iloc[:,0]
        x = concat.iloc[:,1:].values
        model.fit(x, y)
        resi = pd.Series(y - model.predict(x), index=y.index)        
        res_residual[date] = resi
    df_resi = pd.DataFrame(res_residual).T
    return df_resi 

In [19]:
def zscore(x):
    """
    标准化
    """
    return (x - np.nanmean(x)) / np.nanstd(x)

In [20]:
def extreme_median(x):
    """
    5倍中位数去极值
    """
    md = np.nanmedian(x)
    mad = np.nanmedian(abs(x - md))
    return np.clip(x, md-5*mad, md+5*mad)

In [14]:
def del_trad_pct(stock,q):
    """
    STR剔除换手率+换手变化率
    """
    try:
    #     model = linear_model.LinearRegression()
        s_str = tov_str[stock]
        s_str = (s_str - np.nanmean(s_str)) / np.nanstd(s_str)
        s_pct = tov_pct[stock]
        s_pct = (s_pct - np.nanmean(s_pct)) / np.nanstd(s_pct)
        s_trad = tov_trad[stock]
        s_trad = (s_trad - np.nanmean(s_trad)) / np.nanstd(s_trad)
        idx_a = s_trad[s_trad.isnull()].index
        idx_b = s_pct[s_pct.isnull()].index
    #     idx_c = s_str[s_str.isnull()].index
        total = list(idx_a.union(idx_b))
    #     print(len(total))
        if len(total) == 1339:
            resid = np.full(len(total),np.nan)
        else:
            s_pct[total]= np.nan
            s_trad[total]=np.nan
            s_str[total] = np.nan
            sub = pd.concat([s_pct,s_trad],axis =1)
#             print(sub.shape)
            rg = LinearRegression().fit(sub.dropna(),s_str.dropna())
            y_pred =sub.dot(rg.coef_.transpose())+ rg.intercept_
    #         print(y_pred)
            resid = s_str-y_pred
            resid = resid.values
#         print(resid)
        return q.put({stock:resid})
    except Exception as e:
        print(e, stock)
        return

In [31]:
q = multiprocessing.Manager().Queue()
param = []
for stock in tov.columns:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =6) as pool:
    pool.starmap(std_mean,param)
    
#extract
elements = []
while q.qsize():
    elements.append(q.get())

In [32]:
d = pd.DataFrame(np.empty((len(tov),tov.shape[1]),dtype=object),columns = tov.columns, index = tov.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    d[stock]=val

In [33]:
d.to_csv("tov_std_mean.csv")

In [33]:
d[tov_str.columns].to_csv("pct_tov.csv")

In [10]:
d

,000001.XSHE,000002.XSHE,000003.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,605162.XSHG,688226.XSHG,301039.XSHE,688789.XSHG,605020.XSHG,688087.XSHG,688305.XSHG,605028.XSHG,688038.XSHG,688799.XSHG
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
